In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Conv3d layer input shape:
# N -> number of sequences (mini batch)
# Cin -> number of channels (3 for rgb)
# D -> Number of images in a sequence
# H -> Height of one image in the sequence
# W -> Width of one image in the sequence


import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier
from flash.image.classification.integrations.baal import ActiveLearningDataModule, ActiveLearningLoop

In [13]:
head = nn.Sequential(
            #nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
            #nn.Flatten(),
            #nn.Linear(in_features=16384, out_features=4096),
            #nn.ReLU(), 
            #nn.Dropout(0.25),
            #nn.Linear(in_features=4096, out_features=256),
            #nn.ReLU(), 
            nn.Dropout(0.25),
            nn.Linear(in_features=512, out_features=3),
        )
model = ImageClassifier(backbone="vgg16", head=head, num_classes=3)
model.adapter.backbone = model.adapter.backbone[2:]

Using 'vgg16' provided by PyTorch/torchvision (https://github.com/pytorch/vision).


In [14]:
model(torch.rand((5, 64, 512, 512)))

tensor([[-0.0007, -0.0125, -0.1118],
        [ 0.0047, -0.0447, -0.1289],
        [ 0.0510, -0.0130, -0.0777],
        [ 0.0082, -0.0617, -0.0977],
        [ 0.0290, -0.0455, -0.1132]], grad_fn=<AddmmBackward0>)

In [17]:
from torchsummary import summary
#from src.models.conv_net import MedicalNet
#model = MedicalNet(path_to_weights="resnet_10.pth", device="cuda")
model.cuda()
summary(model.adapter, (64, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]          36,928
              ReLU-2         [-1, 64, 256, 256]               0
         MaxPool2d-3         [-1, 64, 128, 128]               0
            Conv2d-4        [-1, 128, 128, 128]          73,856
              ReLU-5        [-1, 128, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]         147,584
              ReLU-7        [-1, 128, 128, 128]               0
         MaxPool2d-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
             ReLU-10          [-1, 256, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         590,080
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [20]:
trainer = flash.Trainer(max_epochs=3)
datamodule = ActiveLearningDataModule(
    train_datset,
    initial_num_labels=5,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Bastian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3361: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


AttributeError: 'CustomFileDataset' object has no attribute 'num_classes'

In [15]:
import sys; sys.path.insert(0, "..") 
import os
import random
import numpy as np

def seed_all(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

In [18]:
import src.visualization.utils as vis
from torchvision import transforms
from src.dataset.dataset import CustomDataset, CustomFileDataset, collate_fn_padd
from src.dataset.utils import get_active_learning_datasets, MyRotationTransform
import torch

seed_all(123)

dataset = CustomDataset(data_path="../data/NPY/volumes/",
                             target_path="../data/NPY/labels/")
train_indices, val_indices, test_indices, _= get_active_learning_datasets(dataset, split=[0.7, 0.1, 0.2])

train_transform = transforms.Compose([transforms.Resize(512//2),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.1), scale=(0.5, 0.75)),
                                        #transforms.ColorJitter(brightness=.5, hue=.3),
                                      ])

test_transform = transforms.Compose([transforms.Resize(512),
                                    ])
train_datset = CustomFileDataset([dataset.data_paths[idx] for idx in list(train_indices)],
                                        [dataset.targets[idx] for idx in list(train_indices)],
                                            train_transform)

test_datset = CustomFileDataset([dataset.data_paths[idx] for idx in list(train_indices)],
                                        [dataset.targets[idx] for idx in list(train_indices)],
                                            train_transform)

CP sample len 1496
NCP sample len 1218
Normal sample len 472
size train files: 990
size test files: 282
size val files: 141
size initial pool files: 3


In [ ]:
dataset_module = ActiveLearningDataModule(initial_num_labels=5, )

In [ ]:
train_datset[0][0].shape

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_datset,
                            num_workers=4,
                            batch_size=2,
                            pin_memory=True,
                            shuffle=False,
                            collate_fn=collate_fn_padd)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0]
print(img.shape)
label = train_labels[0]
vis.plot_from_volume_tensor(img, label, n_samples=10)
print(f"Label: {label}")